In [ ]:
import pandas as pd
import numpy as np
from statistics import mean

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/oreilly_january/huge_df.csv')

In [ ]:
df['word_count'] = df['snippet'].apply(lambda x: len(x.split(' ')))

In [ ]:
df['charecter_count'] = df['snippet'].apply(lambda x: len(x))

In [ ]:
df

,identifier,date_published,Title,Status,Author,chapter_name,snippet,Snippet_ID,custom_id,result,Example_A,Example_B,Example_C,Example_D,Answer,Permutation_Number,Formatted Author,word_count,charecter_count
0,9781098101381,2021-09-14,97 Things Every Information Security Professio...,public,"Morillo, Christina",ch01,Learn with a community. I’ve personally found ...,0,0,"{'type': 'succeeded', 'message': {'id': 'msg_0...",Learn with a community. I’ve personally found ...,Connect with fellow learners. From my experien...,Join a learning group. Based on my observation...,Engage in collaborative learning. My personal ...,A,1,Christina Morillo,92,576
1,9781098101381,2021-09-14,97 Things Every Information Security Professio...,public,"Morillo, Christina",ch01,Learn with a community. I’ve personally found ...,0,0,"{'type': 'succeeded', 'message': {'id': 'msg_0...",Learn with a community. I’ve personally found ...,Connect with fellow learners. From my experien...,Engage in collaborative learning. My personal ...,Join a learning group. Based on my observation...,A,2,Christina Morillo,92,576
2,9781098101381,2021-09-14,97 Things Every Information Security Professio...,public,"Morillo, Christina",ch01,Learn with a community. I’ve personally found ...,0,0,"{'type': 'succeeded', 'message': {'id': 'msg_0...",Learn with a community. I’ve personally found ...,Join a learning group. Based on my observation...,Connect with fellow learners. From my experien...,Engage in collaborative learning. My personal ...,A,3,Christina Morillo,92,576
3,9781098101381,2021-09-14,97 Things Every Information Security Professio...,public,"Morillo, Christina",ch01,Learn with a community. I’ve personally found ...,0,0,"{'type': 'succeeded', 'message': {'id': 'msg_0...",Learn with a community. I’ve personally found ...,Join a learning group. Based on my observation...,Engage in collaborative learning. My personal ...,Connect with fellow learners. From my experien...,A,4,Christina Morillo,92,576
4,9781098101381,2021-09-14,97 Things Every Information Security Professio...,public,"Morillo, Christina",ch01,Learn with a community. I’ve personally found ...,0,0,"{'type': 'succeeded', 'message': {'id': 'msg_0...",Learn with a community. I’ve personally found ...,Engage in collaborative learning. My personal ...,Connect with fellow learners. From my experien...,Join a learning group. Based on my observation...,A,5,Christina Morillo,92,576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335083,9781098145590,2024-06-17,Web Accessibility Cookbook,private,"Matuzovic, Manuel",ch13,Testing tools usually only report accessibilit...,13961,13961,"{'type': 'succeeded', 'message': {'id': 'msg_0...",Accessibility validation programs generally fo...,Testing tools usually only report accessibilit...,Most accessibility checkers only flag issues b...,Accessibility evaluation software typically id...,B,20,Manuel Matuzovic,102,633
335084,9781098145590,2024-06-17,Web Accessibility Cookbook,private,"Matuzovic, Manuel",ch13,Testing tools usually only report accessibilit...,13961,13961,"{'type': 'succeeded', 'message': {'id': 'msg_0...",Accessibility validation programs generally fo...,Accessibility evaluation software typically id...,Testing tools usually only report accessibilit...,Most accessibility checkers only flag issues b...,C,21,Manuel Matuzovic,102,633
335085,9781098145590,2024-06-17,Web Accessibility Cookbook,private,"Matuzovic, Manuel",ch13,Testing tools usually only report accessibilit...,13961,13961,"{'type': 'succeeded', 'message': {'id': 'msg_0...",Accessibility validation programs generally fo...,Accessibility evaluation software typically id...,Most accessibility checkers only flag issues b...,Testing tools usually only report accessibilit...,D,22,Manuel Matuzovic,102,633
335086,9781098145590,2024-06-17,Web Accessibility Cookbook,private,"Matuzovic, Manuel",ch13,Testing tools usually only report accessibilit...,13961,13961,"{'type': 'succeeded', 'mes

In [ ]:
import pandas as pd
from collections import Counter
import re

public = df[df['Status'] == 'public']
private = df[df['Status'] == 'private']


def extract_common_phrases(df, column_name='snippet', min_words=5, top_n=5):
    """
    Extracts the most common phrases with at least `min_words` words from the specified column of a DataFrame.

    Args:
    - df (pd.DataFrame): The input DataFrame.
    - column_name (str): The name of the column containing text data.
    - min_words (int): The minimum number of words a phrase must have to be considered.
    - top_n (int): The number of top phrases to return.

    Returns:
    - A DataFrame containing the most common phrases and their counts.
    """
    sub_df = df[df['Permutation_Number'] == 1]
    # Combine all text from the specified column
    all_text = ' '.join(sub_df[column_name].astype(str).tolist())


    # Tokenize into sentences and remove non-word characters
    sentences = re.split(r'[.!?]', all_text)
    sentences = [re.sub(r'[^a-zA-Z\s]', '', sentence).lower() for sentence in sentences]

    # Extract all possible phrases with at least min_words
    phrases = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) >= min_words:
            for i in range(len(words) - min_words + 1):
                phrase = ' '.join(words[i:i + min_words])
                phrases.append(phrase)

    # Count the most common phrases
    phrase_counts = Counter(phrases)

    # Get the top_n most common phrases
    most_common_phrases = phrase_counts.most_common(top_n)

    # Convert to DataFrame for display
    result_df = pd.DataFrame(most_common_phrases, columns=['Phrase', 'Count'])

    return result_df


extract_common_phrases(public, column_name='snippet', min_words=3, top_n=5)

,Phrase,Count
0,in this chapter,138
1,as well as,115
2,one of the,99
3,be able to,89
4,a lot of,85


In [ ]:
3312 /24

138.0

In [ ]:
449 *24

10776

In [ ]:
extract_common_phrases(private, column_name='snippet', min_words=3, top_n=5)

,Phrase,Count
0,as well as,455
1,one of the,449
2,be able to,444
3,the number of,436
4,you want to,399


In [ ]:
import re
from collections import Counter

# Convert 'date_published' to datetime
df['date_published'] = pd.to_datetime(df['date_published'], errors='coerce')

# Define the training period end date (for "in-dataset" vs "out of dataset")
training_period_end_date = pd.Timestamp('2023-08-01')

# Filter the data into 'in-dataset' and 'out of dataset'
in_dataset = df[df['date_published'] <= training_period_end_date]
out_of_dataset = df[df['date_published'] > training_period_end_date]

# Calculate descriptive statistics for each subset
def calculate_descriptive_stats(df, label):
    num_chapters = len(df)
    avg_charecter_length = df['snippet'].apply(lambda x: len(x)).mean()
    avg_date_published = df['date_published'].mean()
    return {
        'Label': label,
        'Sample Size (n)': num_chapters,
        # 'Average Charecter Length': round(avg_charecter_length),
        'Average Word Count': round(df['word_count'].mean()),
        'Average Date Published': avg_date_published.strftime('%m/%d/%Y')
    }

# Descriptive statistics for the entire dataset
total_stats = calculate_descriptive_stats(df, 'Total')

# Descriptive statistics for public and private
public_stats = calculate_descriptive_stats(df[df['Status'] == 'public'], 'Public')
private_stats = calculate_descriptive_stats(df[df['Status'] == 'private'], 'Private')

# Descriptive statistics for 'in-dataset' and 'out of dataset'
in_dataset_stats = calculate_descriptive_stats(in_dataset, 'In-Dataset')
out_of_dataset_stats = calculate_descriptive_stats(out_of_dataset, 'Out-of-Dataset')

# Combine results into a DataFrame for display
descriptive_stats_df = pd.DataFrame([
    total_stats,
    public_stats,
    private_stats,
    in_dataset_stats,
    out_of_dataset_stats
])

# Display the descriptive statistics
# import ace_tools as tools; tools.display_dataframe_to_user(name="Descriptive Statistics", dataframe=descriptive_stats_df)
descriptive_stats_df


,Label,Sample Size (n),Average Word Count,Average Date Published
0,Total,335088,112,05/11/2022
1,Public,59952,112,05/16/2022
2,Private,275136,112,05/10/2022
3,In-Dataset,260160,113,10/24/2021
4,Out-of-Dataset,74928,110,03/30/2024


In [ ]:
latex = descriptive_stats_df.to_latex(index=False, escape=False)
print(latex)



\begin{tabular}{lrrr}
\toprule
Label & Sample Size (n) & Average Charecter Length & Average Word Count \\
\midrule
Total & 1202 & 342 & 56 \\
Public & 581 & 353 & 57 \\
Private & 621 & 331 & 54 \\
In-Dataset & 784 & 358 & 58 \\
Out-of-Dataset & 418 & 310 & 51 \\
\bottomrule
\end{tabular}



In [ ]:
import json

In [ ]:
titles = df['title'].unique()
json.dump(titles.tolist(), open('titles.json', 'w'))